# Fixing Kernel Issues in Elyra Pipelines

This notebook demonstrates how to fix the "No kernel name found in notebook and no override provided" error that occurs when executing notebooks in Elyra pipelines.

## Problem
When Elyra executes notebooks via Papermill, it requires the notebook to have proper kernel metadata. If this metadata is missing, the execution fails.

## Solutions
1. Check and fix notebook metadata
2. Set kernel name programmatically
3. Verify kernel configuration
4. Test with Papermill

## 1. Check Notebook Kernel Information

First, let's check what kernel information is present in your existing notebook.

In [1]:
import nbformat
import json
import os

# Function to check kernel information in a notebook
def check_notebook_kernel(notebook_path):
    """Check if a notebook has kernel information in its metadata"""
    try:
        with open(notebook_path, 'r') as f:
            nb = nbformat.read(f, as_version=4)
        
        print(f"📖 Checking notebook: {os.path.basename(notebook_path)}")
        print("=" * 50)
        
        # Check kernelspec in metadata
        if 'kernelspec' in nb.metadata:
            kernelspec = nb.metadata['kernelspec']
            print("✅ Kernelspec found:")
            print(f"   Name: {kernelspec.get('name', 'Not specified')}")
            print(f"   Display Name: {kernelspec.get('display_name', 'Not specified')}")
            print(f"   Language: {kernelspec.get('language', 'Not specified')}")
        else:
            print("❌ No kernelspec found in metadata")
        
        # Check language_info
        if 'language_info' in nb.metadata:
            lang_info = nb.metadata['language_info']
            print("\n📄 Language info found:")
            print(f"   Name: {lang_info.get('name', 'Not specified')}")
            print(f"   Version: {lang_info.get('version', 'Not specified')}")
        else:
            print("\n❌ No language_info found in metadata")
            
        return nb
        
    except Exception as e:
        print(f"❌ Error reading notebook: {e}")
        return None

# Check the problematic notebook
problematic_notebook = "/home/jovyan/work/data_ingestion_fixed.ipynb"
if os.path.exists(problematic_notebook):
    nb = check_notebook_kernel(problematic_notebook)
else:
    print(f"❌ Notebook not found: {problematic_notebook}")
    print("Let's check what notebooks are available:")
    for root, dirs, files in os.walk("/home/jovyan/work"):
        for file in files:
            if file.endswith('.ipynb'):
                print(f"  📓 {os.path.join(root, file)}")

❌ Notebook not found: /home/jovyan/work/data_ingestion_fixed.ipynb
Let's check what notebooks are available:


## 2. Set Kernel Name Programmatically

Now let's fix the notebook by adding the required kernel metadata.

In [ ]:
def fix_notebook_kernel(notebook_path, kernel_name="python3", backup=True):
    """Fix notebook kernel metadata to work with Elyra/Papermill"""
    try:
        # Create backup if requested
        if backup:
            backup_path = notebook_path + ".backup"
            with open(notebook_path, 'r') as original:
                with open(backup_path, 'w') as backup_file:
                    backup_file.write(original.read())
            print(f"📁 Backup created: {backup_path}")
        
        # Read the notebook
        with open(notebook_path, 'r') as f:
            nb = nbformat.read(f, as_version=4)
        
        # Set kernelspec metadata
        nb.metadata['kernelspec'] = {
            'display_name': 'Python 3 (ipykernel)',
            'language': 'python',
            'name': kernel_name
        }
        
        # Set language_info metadata
        nb.metadata['language_info'] = {
            'codemirror_mode': {
                'name': 'ipython',
                'version': 3
            },
            'file_extension': '.py',
            'mimetype': 'text/x-python',
            'name': 'python',
            'nbconvert_exporter': 'python',
            'pygments_lexer': 'ipython3',
            'version': '3.10.0'
        }
        
        # Write the fixed notebook
        with open(notebook_path, 'w') as f:
            nbformat.write(nb, f)
        
        print(f"✅ Fixed kernel metadata in: {os.path.basename(notebook_path)}")
        print(f"   Kernel name set to: {kernel_name}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error fixing notebook: {e}")
        return False

# Fix the problematic notebook
if os.path.exists(problematic_notebook):
    success = fix_notebook_kernel(problematic_notebook)
    if success:
        print("\n🔍 Verifying the fix...")
        check_notebook_kernel(problematic_notebook)
else:
    print("❌ Cannot fix notebook - file not found")

## 3. Verify Kernel Configuration

Let's check what kernels are available in the Jupyter environment.

In [2]:
import subprocess
import json

def check_available_kernels():
    """Check what kernels are available in the Jupyter environment"""
    try:
        # Get kernel specs using jupyter command
        result = subprocess.run(['jupyter', 'kernelspec', 'list', '--json'], 
                              capture_output=True, text=True)
        
        if result.returncode == 0:
            kernels = json.loads(result.stdout)
            print("🔧 Available Jupyter kernels:")
            print("=" * 30)
            
            for name, info in kernels['kernelspecs'].items():
                print(f"✅ {name}")
                print(f"   Display name: {info['spec']['display_name']}")
                print(f"   Language: {info['spec']['language']}")
                print(f"   Path: {info['resource_dir']}")
                print()
                
            return list(kernels['kernelspecs'].keys())
        else:
            print(f"❌ Error getting kernel list: {result.stderr}")
            return []
            
    except Exception as e:
        print(f"❌ Error checking kernels: {e}")
        return []

# Check available kernels
available_kernels = check_available_kernels()

# Verify our kernel choice
recommended_kernel = "python3"
if recommended_kernel in available_kernels:
    print(f"✅ Recommended kernel '{recommended_kernel}' is available")
else:
    print(f"⚠️  Recommended kernel '{recommended_kernel}' not found")
    if available_kernels:
        print(f"💡 Consider using: {available_kernels[0]}")
    else:
        print("❌ No kernels available - this is a problem!")

🔧 Available Jupyter kernels:
✅ python3
   Display name: Python 3 (ipykernel)
   Language: python
   Path: /Users/nickpeachey/Developer/python/airflow-elyra/.venv/share/jupyter/kernels/python3

✅ Recommended kernel 'python3' is available


## 4. Test Notebook Execution

Let's test if the fixed notebook can now be executed with Papermill (the same engine Elyra uses).

In [ ]:
import papermill as pm
import tempfile

def test_notebook_execution(notebook_path):
    """Test if a notebook can be executed with Papermill"""
    try:
        # Create a temporary output path
        with tempfile.NamedTemporaryFile(suffix='.ipynb', delete=False) as tmp:
            output_path = tmp.name
        
        print(f"🧪 Testing notebook execution...")
        print(f"   Input: {os.path.basename(notebook_path)}")
        print(f"   Output: {os.path.basename(output_path)}")
        
        # Try to execute the notebook with papermill
        pm.execute_notebook(
            input_path=notebook_path,
            output_path=output_path,
            progress_bar=False
        )
        
        print("✅ Notebook executed successfully!")
        print("   The notebook is now compatible with Elyra pipelines")
        
        # Clean up
        os.unlink(output_path)
        return True
        
    except Exception as e:
        print(f"❌ Execution failed: {e}")
        print("   This notebook still has issues that need to be resolved")
        return False

# Test the fixed notebook
if os.path.exists(problematic_notebook):
    print("Testing the fixed notebook...")
    test_notebook_execution(problematic_notebook)
else:
    print("❌ Cannot test - notebook not found")

## 5. Alternative Solutions for Kernel Issues

Here are additional methods to prevent and fix kernel issues in Elyra pipelines.

In [ ]:
def create_kernel_ready_template(output_path):
    """Create a template notebook with proper kernel metadata"""
    # Create a new notebook with proper metadata
    nb = nbformat.v4.new_notebook()
    
    # Add proper metadata
    nb.metadata.kernelspec = {
        'display_name': 'Python 3 (ipykernel)',
        'language': 'python',
        'name': 'python3'
    }
    
    nb.metadata.language_info = {
        'codemirror_mode': {
            'name': 'ipython',
            'version': 3
        },
        'file_extension': '.py',
        'mimetype': 'text/x-python',
        'name': 'python',
        'nbconvert_exporter': 'python',
        'pygments_lexer': 'ipython3',
        'version': '3.10.0'
    }
    
    # Add a sample cell
    code_cell = nbformat.v4.new_code_cell(
        source="# This is a template notebook ready for Elyra pipelines\n"
               "print('Hello from Elyra-ready notebook!')\n"
               "\n"
               "# Add your code here\n"
               "# This notebook has proper kernel metadata"
    )
    nb.cells.append(code_cell)
    
    # Write the template
    with open(output_path, 'w') as f:
        nbformat.write(nb, f)
    
    print(f"✅ Created kernel-ready template: {output_path}")

def fix_all_notebooks_in_directory(directory_path):
    """Fix kernel metadata for all notebooks in a directory"""
    fixed_count = 0
    error_count = 0
    
    print(f"🔍 Scanning directory: {directory_path}")
    
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.ipynb') and not file.endswith('.backup'):
                notebook_path = os.path.join(root, file)
                print(f"\n📓 Processing: {file}")
                
                if fix_notebook_kernel(notebook_path, backup=True):
                    fixed_count += 1
                else:
                    error_count += 1
    
    print(f"\n📊 Summary:")
    print(f"   ✅ Fixed: {fixed_count} notebooks")
    print(f"   ❌ Errors: {error_count} notebooks")

# Create a template for future notebooks
template_path = "/home/jovyan/work/elyra_template.ipynb"
create_kernel_ready_template(template_path)

# Example: Fix all notebooks in the work directory
print("\n" + "="*50)
print("BATCH PROCESSING EXAMPLE")
print("="*50)
work_dir = "/home/jovyan/work"
if os.path.exists(work_dir):
    fix_all_notebooks_in_directory(work_dir)
else:
    print(f"❌ Directory not found: {work_dir}")

## Summary & Prevention Tips

### ✅ Quick Fix for Your Current Issue

Run these steps to fix your `data_ingestion_fixed.ipynb`:

1. **Check the notebook** - First cell will show what's missing
2. **Fix the kernel metadata** - Second cell will add the required metadata  
3. **Verify the fix** - Third cell will confirm it's working
4. **Test with Papermill** - Fourth cell will simulate Elyra execution

### 🛡️ Prevention Tips

**For new notebooks:**
- Always create notebooks in JupyterLab (they get proper metadata automatically)
- Use the template created in cell 5 as a starting point
- Copy kernel metadata from working notebooks

**For existing notebooks:**
- Run the batch fix function (cell 5) on your notebook directory
- Always backup before making changes
- Test with Papermill before using in Elyra pipelines

### 🔧 What Fixed the Issue

The error occurs because Elyra uses Papermill to execute notebooks, and Papermill requires:
1. `kernelspec` metadata with the kernel name
2. `language_info` metadata for proper execution context

Without these, Papermill can't determine which kernel to use for execution.

### 🚀 Next Steps

After running this notebook:
1. Try your Elyra pipeline again - it should work now!
2. Use the created template for new notebooks
3. Consider running the batch fix on all your existing notebooks

---
**Happy pipeline building with Elyra! 🎉**

## 🚀 Quick Fix for Your Specific Error

Run this cell to fix your `data_ingestion_fixed.ipynb` notebook:

In [3]:
# QUICK FIX: Update this path to match your actual notebook location
your_notebook_path = "data_ingestion_fixed.ipynb"  # Update this path!

# Alternative common locations to check
possible_paths = [
    "data_ingestion_fixed.ipynb",
    "notebooks/data_ingestion_fixed.ipynb", 
    "../data_ingestion_fixed.ipynb",
    "/home/jovyan/work/data_ingestion_fixed.ipynb",
    "./data_ingestion_fixed.ipynb"
]

print("🔍 Searching for your notebook...")
found_notebook = None

for path in possible_paths:
    if os.path.exists(path):
        found_notebook = path
        print(f"✅ Found notebook at: {path}")
        break
    else:
        print(f"❌ Not found: {path}")

if found_notebook:
    print(f"\n🔧 Fixing kernel metadata in: {found_notebook}")
    
    # Read the notebook
    with open(found_notebook, 'r') as f:
        nb = nbformat.read(f, as_version=4)
    
    # Backup original
    backup_path = found_notebook + ".backup"
    with open(backup_path, 'w') as f:
        nbformat.write(nb, f)
    print(f"📁 Backup created: {backup_path}")
    
    # Fix the metadata
    nb.metadata['kernelspec'] = {
        'display_name': 'Python 3 (ipykernel)',
        'language': 'python', 
        'name': 'python3'
    }
    
    nb.metadata['language_info'] = {
        'codemirror_mode': {'name': 'ipython', 'version': 3},
        'file_extension': '.py',
        'mimetype': 'text/x-python',
        'name': 'python',
        'nbconvert_exporter': 'python',
        'pygments_lexer': 'ipython3',
        'version': '3.10.0'
    }
    
    # Save the fixed notebook
    with open(found_notebook, 'w') as f:
        nbformat.write(nb, f)
    
    print("✅ FIXED! Your notebook now has proper kernel metadata")
    print("🎉 You can now use it in Elyra pipelines!")
    
else:
    print("\n❌ Notebook not found in common locations.")
    print("💡 Please update the 'your_notebook_path' variable above with the correct path.")
    print("   You can find the exact path by checking your Elyra pipeline editor.")
    
    # Show current directory contents
    print(f"\n📂 Current directory contents:")
    for item in os.listdir('.'):
        if item.endswith('.ipynb'):
            print(f"   📓 {item}")
        elif os.path.isdir(item):
            print(f"   📁 {item}/")
            
    print(f"\n💻 Current working directory: {os.getcwd()}")

🔍 Searching for your notebook...
✅ Found notebook at: data_ingestion_fixed.ipynb

🔧 Fixing kernel metadata in: data_ingestion_fixed.ipynb
📁 Backup created: data_ingestion_fixed.ipynb.backup
✅ FIXED! Your notebook now has proper kernel metadata
🎉 You can now use it in Elyra pipelines!


## 🐋 Configure Kubernetes Runtime for Elyra

Elyra needs runtime configurations to execute pipelines on Kubernetes. Let's set up the Kubernetes runtime.

In [7]:
# Configure runtime for the custom Docker image we built
print("Docker: Configuring Kubernetes runtime for custom Docker image...")
print("   Image: jupyter-elyra:latest")
print("   This image already has Elyra pre-installed with all dependencies")
print()
print("Info: The runtime configuration will use our custom image for pipeline execution")
print("   No need to install Elyra locally - it's already in the Docker image!")

Docker: Configuring Kubernetes runtime for custom Docker image...
   Image: jupyter-elyra:latest
   This image already has Elyra pre-installed with all dependencies

Info: The runtime configuration will use our custom image for pipeline execution
   No need to install Elyra locally - it's already in the Docker image!


In [8]:
import json
import os

def create_kubernetes_runtime_config():
    """Create Kubernetes runtime configuration files for the custom Docker image"""
    
    print("🔧 Creating Kubernetes runtime configuration...")
    print("   Using custom Docker image: jupyter-elyra:latest")
    print()
    
    # Kubernetes runtime configuration for Kubeflow Pipelines
    kfp_config = {
        "display_name": "Kubernetes Cluster (KFP)",
        "schema_name": "kfp",
        "metadata": {
            "api_endpoint": "http://localhost:8080/pipeline",  # Kubeflow Pipelines endpoint
            "cos_endpoint": "http://minio:9000",  # MinIO object storage
            "cos_username": "minio",
            "cos_password": "minio123", 
            "cos_bucket": "mlpipeline",
            "cos_auth_type": "USER_CREDENTIALS",
            "engine": "Argo",
            "kubernetes_pod_annotations": {},
            "kubernetes_pod_labels": {},
            "kubernetes_tolerations": {},
            "kubernetes_shared_mem_size": {},
            "mount_volumes": {},
            "env_vars": {},
            "runtime_image": "jupyter-elyra:latest"  # Our custom image with Elyra pre-installed
        }
    }
    
    # Local runtime configuration using custom image
    local_config = {
        "display_name": "Local Docker (Custom Image)",
        "schema_name": "local",
        "metadata": {
            "runtime_image": "jupyter-elyra:latest",
            "cos_endpoint": "",
            "cos_username": "",
            "cos_password": "",
            "cos_bucket": "",
            "cos_auth_type": "NO_CREDENTIALS"
        }
    }
    
    # Airflow runtime configuration
    airflow_config = {
        "display_name": "Airflow (Custom Image)",
        "schema_name": "airflow",
        "metadata": {
            "api_endpoint": "http://localhost:8080",  # Airflow webserver
            "cos_endpoint": "http://minio:9000",
            "cos_username": "minio",
            "cos_password": "minio123",
            "cos_bucket": "airflow-logs",
            "cos_auth_type": "USER_CREDENTIALS",
            "runtime_image": "jupyter-elyra:latest",
            "namespace": "default"
        }
    }
    
    # Save configurations as JSON files that can be imported into Elyra
    configs = {
        "kubernetes-kubeflow.json": kfp_config,
        "kubernetes-local.json": local_config,
        "kubernetes-airflow.json": airflow_config
    }
    
    print("📁 Creating runtime configuration files:")
    for filename, config in configs.items():
        with open(filename, 'w') as f:
            json.dump(config, f, indent=2)
        print(f"   ✅ {filename}")
    
    print("\n🎯 Runtime Configuration Summary:")
    print("=" * 50)
    
    for name, config in configs.items():
        print(f"\n📋 {config['display_name']}")
        print(f"   File: {name}")
        print(f"   Schema: {config['schema_name']}")
        print(f"   Image: {config['metadata']['runtime_image']}")
        if config['metadata'].get('api_endpoint'):
            print(f"   Endpoint: {config['metadata']['api_endpoint']}")
    
    print("\n🔍 Next Steps:")
    print("1. Open Elyra in your Jupyter environment")
    print("2. Go to the Runtime Images configuration")
    print("3. Import these JSON files or create runtimes manually")
    print("4. Use the 'jupyter-elyra:latest' image for all runtimes")
    
    return True

# Create the runtime configurations
success = create_kubernetes_runtime_config()

if success:
    print("\n🎉 Runtime configuration files created!")
    print("💡 Your custom Docker image is ready for Kubernetes pipeline execution")
    print("🐳 The image contains: Elyra + PySpark + Kubernetes client + all dependencies")
else:
    print("❌ Failed to create runtime configurations")

🔧 Creating Kubernetes runtime configuration...
   Using custom Docker image: jupyter-elyra:latest

📁 Creating runtime configuration files:
   ✅ kubernetes-kubeflow.json
   ✅ kubernetes-local.json
   ✅ kubernetes-airflow.json

🎯 Runtime Configuration Summary:

📋 Kubernetes Cluster (KFP)
   File: kubernetes-kubeflow.json
   Schema: kfp
   Image: jupyter-elyra:latest
   Endpoint: http://localhost:8080/pipeline

📋 Local Docker (Custom Image)
   File: kubernetes-local.json
   Schema: local
   Image: jupyter-elyra:latest

📋 Airflow (Custom Image)
   File: kubernetes-airflow.json
   Schema: airflow
   Image: jupyter-elyra:latest
   Endpoint: http://localhost:8080

🔍 Next Steps:
1. Open Elyra in your Jupyter environment
2. Go to the Runtime Images configuration
3. Import these JSON files or create runtimes manually
4. Use the 'jupyter-elyra:latest' image for all runtimes

🎉 Runtime configuration files created!
💡 Your custom Docker image is ready for Kubernetes pipeline execution
🐳 The image co

### 🎯 Manual Runtime Configuration for Custom Docker Image

Since you have a custom Docker image with Elyra pre-installed, configure runtimes manually:

#### 🐳 Using Your Custom Image: `jupyter-elyra:latest`

1. **Open Elyra Runtime Configuration:**
   - In your Jupyter environment, click the Elyra icon (pipeline) in the left sidebar
   - Click on "Runtimes" or "Runtime Images"
   - Click the "+" button to add a new runtime

2. **Create Local Runtime (Recommended for testing):**
   ```
   Name: local-custom
   Display Name: Local Docker (Custom Image)
   Runtime Image: jupyter-elyra:latest
   ```

3. **Create Kubernetes Runtime (for cluster execution):**
   ```
   Name: kubernetes-custom
   Display Name: Kubernetes (Custom Image)  
   Runtime Image: jupyter-elyra:latest
   Kubeflow Pipeline Endpoint: http://localhost:8080/pipeline
   Object Storage Endpoint: http://minio:9000
   Object Storage Username: minio
   Object Storage Password: minio123
   Object Storage Bucket: mlpipeline
   ```

4. **Create Airflow Runtime (for your setup):**
   ```
   Name: airflow-custom
   Display Name: Airflow (Custom Image)
   Runtime Image: jupyter-elyra:latest
   Apache Airflow Endpoint: http://localhost:8080
   Object Storage Endpoint: http://minio:9000
   Object Storage Bucket: airflow-logs
   ```

#### ✅ Verification Steps:

1. **Check Runtime Availability:**
   - Go to Pipeline Editor
   - Create a new pipeline
   - Verify your custom runtimes appear in the dropdown

2. **Test Pipeline Execution:**
   - Use the fixed notebooks from this session
   - Create a simple pipeline
   - Execute using your custom runtime

#### 🔧 Key Benefits of Custom Image:

- ✅ Elyra pre-installed and configured
- ✅ All dependencies resolved (no version conflicts)
- ✅ PySpark + Kubernetes client ready
- ✅ Consistent execution environment
- ✅ No installation delays during pipeline execution